be really clear about what's going into repaired tables: 
- repaired
- already OK
- unrepairable

Only ~ 215k repaired `env_package`s ?!
- yeah, there just aren't that many annotated in the biosample metadata


- some of this code should probably be moved back into the package
    - splitting and repiping 

- read in previous SSSOMs and don't re-map? or at least double check?

- don't forget to consult "model" field in biosample table too
    - for scoping, in addition to en_package and taxonomy_id?

- what to do about hits against obsolete classes... does OLS provide a mechanism to avoid?
- be careful with GAZ matches
- avoid IDless, OMIT, NCIT, MICRO, BTO

- is it worth trimming and concatenating "x1", "x2", "x3", where x is a mappable term? numbers may be low.
    - example = XXX

- reconcile "x biome" and "x" mappings
- take note of non-local salvage hits
    - like XXX
    - small/large (freshwater) lake?
    - include native labels in the review frame, in addition to local?
    - should use rdftab, not additional OLS calls
    

- switch from hard-coded `env_package_overrides` to something like SSSOM?
- mappings can come from merging strategies, first pass OLS or salvage OLS
- reflect those methods in SSSOM
- add string distance/confidence
    - multiple columns for multiple methods?

review these SSSOM columns

```
    "subject_category": [biosample_col_to_map] * results_rows,
    "predicate_id": ["skos:relatedMatch"] * results_rows,
    "match_type": ["Lexical"] * results_rows,
    "creator_id": ["https://github.com/turbomam/scoped-mapping"] * results_rows,
    #     "mapping_tool": ["https://www.ebi.ac.uk/ols/docs/api"] * results_rows,
    #     "confidence": 1 - frame_for_sssom["string_dist"],
```

- are there samples with a host_taxid and a (sample) taxonomy_id from "unclassified"?
- start at uncultured or metagenome taxonomy_id?

- what scoping on or mapping of  "gold" or "path"? WON'T FIND THAT IN NCBI BIOSAMPLE


What to save where? 
- new columns in biosample sqlite? 
- delimited file dumps
- move content from original column and insert the new values into that original column?
- pandas clipboard integration handy for review step


In [1]:
import pandas as pd
import re
import requests as requests
import sqlite3
import string
import urllib
import yaml

import scoped_mapping

from datetime import datetime
from pkg_resources import get_distribution, DistributionNotFound
from strsimpy.cosine import Cosine
from xml.etree import ElementTree
from tdda import rexpy

# Settings required from user:

See repo README for notes on setting up SQLite databases of OBO ontologies with semantic-sql, relation-graph and rdftab

### How should the mapping effort be scoped?
Additional taxonomic filters may be applied below
- soil
- sediment
- plant-associated

In [2]:
scoping_col = "env_package_normalization.EnvPackage"
scoping_value = "sediment"

### What Biosample field should be mapped to ontology classes?
- env_broad_scale
- env_local_scale
- env_medium

In [3]:
biosample_col_to_map = "env_medium"

In [4]:
# where do we require a single ontology and where can we use multiple?
primary_onto_prefix = "ENVO"
# 'gaz',
# need to rank results
# additional_preferred_ontologies = ["ncbitaxon", "obi", "pato", "efo"]
additional_preferred_ontologies = ["ncbitaxon", "po"]

In [5]:
# from https://www.ncbi.nlm.nih.gov/biosample/docs/packages/?format=xml
# see also https://www.ncbi.nlm.nih.gov/biosample/docs/packages/
biosample_packages_file = "../target/biosample_packages.xml"

# from ftp://ftp.ncbi.nlm.nih.gov//biosample/biosample_set.xml.gz
# via harmonized_table.db.gz
# in https://drive.google.com/drive/u/0/folders/1eL0v0stoduahjDpoDJIk3z2pJBAU4b2Y
biosample_sqlite_file = "../target/harmonized_table.db"
ncbitaxon_sqlite_file = "../semantic-sql/db/ncbitaxon.db"

chars_to_whiteout = "._-"
my_query_fields = ""  # OLS weighted default
my_row_req = 3
my_string_dist_arg = 2
my_max_string_dist = 0.1
salvage_max_dist = 0.2

### Settings for manual review of OLS search-based results
Results based on merging Biosample annotations to ontology classes by label or embedded ID are not exported for review at this time

In [6]:
# ols_review_file = "../local/ols_review.tsv"
ols_review_seperator = "\t"

strategy_col = "strategy"
include_col = "include"

first_pass_include_val = True
# first_pass_strategy_val = "env_braod_scale vs envo and gaz @ 0.1"
salvage_include_val = False
# salvage_strategy_val    = "env_braod_scale vs all of ols @ 0.2"

### Settings for SSSOM output

In [7]:
sssom_subject_prefix = biosample_col_to_map + "_" + scoping_col + "_" + scoping_value
sssom_file = "../" + sssom_subject_prefix + "_SSSOM.tsv"

In [8]:
# per_biosample_mapping_results_file = "per_biosample_scoped_mapping_results.tsv"

----

In [9]:
biosample_cnx = sqlite3.connect(biosample_sqlite_file)
ncbitaxon_cnx = sqlite3.connect(ncbitaxon_sqlite_file)
primary_ontology_sqlite_file = (
    "../semantic-sql/db/" + primary_onto_prefix.lower() + ".db"
)
primary_ontology_cnx = sqlite3.connect(primary_ontology_sqlite_file)
additional_preferred_ontologies.insert(0, primary_onto_prefix.lower())
additional_preferred_ontologies_str = ",".join(additional_preferred_ontologies)

In [10]:
q = "select * from id_patterns"
[id_patterns_frame, query_duration] = scoped_mapping.timed_query(q, biosample_cnx)
id_patterns_frame

,ontology,id_pattern
0,BFO,BFO:\d{7}
1,CARO,CARO:\d{7}
2,CHEBI,"CHEBI:\d{4,6}"
3,ENVO,"ENVO:\d{7,8}"
4,FAO,FAO:0000001
5,FOODON,FOODON:\d{8}
6,GO,GO:\d{7}
7,IAO,IAO:\d{7}
8,NCBITaxon,NCBITaxon:\d+
9,OBI,OBI:\d{7}


In [11]:
id_patterns = dict(zip(id_patterns_frame.ontology, id_patterns_frame.id_pattern))
# id_patterns

- env_package = env_package annotation from NCBI Biosample file XXX
- count = number of biosamples using that env_package annotation
- lhs = checklist info
- rhs = potential package info
- extract = potential OBO ID from rhs column (currently harcoded and only looking for ENVO IDs)
- remaining_string = rhs/string, with potential OBO IDs removed
- remaining_tidied = remaining_string with case, whitespace and punctuation normailzastion
- rt_override = some remaining_tidied values can be replaced according to env_package_overrides
- EnvPackage = corresponding de-normalized value from package_dictionary
- is_canonical = false when EnvPackage is NaN

In [12]:
q = "select * from env_package_normalization"
[env_package_normalization, query_duration] = scoped_mapping.timed_query(
    q, biosample_cnx
)

# env_package_normalization

## Get a table of MIxS annotations to be mapped to ontology classes.

Explicitly scope based on normalized package data. These values were set at the top of this notebook.


In [13]:
print(biosample_col_to_map)
print(scoping_col)
print(scoping_value)

env_medium
env_package_normalization.EnvPackage
sediment


**In this case, the scoping includes an inner join requirement for 'unclassified entities'**

In [14]:
q = (
    "select "
    + biosample_col_to_map
    + """, count(*) as count
from
    biosample b
join env_package_normalization on
    b.env_package = env_package_normalization.env_package
inner join biosample_sample_taxon_summary stic on
    b.taxonomy_id = stic.biosample_taxid
where """
    + scoping_col
    + " = '"
    + scoping_value
    + "' group by "
    + biosample_col_to_map
    + """
order by
    count(*) desc"""
)

[mapping_candidates, query_duration] = scoped_mapping.timed_query(q, biosample_cnx)

mapping_candidates

,env_medium,count
0,mud,3917
1,marine sediment,854
2,sediment,702
3,Sediment,450
4,deep marine sediment [ENVO:00002113],185
...,...,...
71,Sequences,1
72,Marine sediment,1
73,Lake Sediment,1
74,Environmental_sample,1


----

## The Biosample format allows for pipe-delimited environmental package lists. 

Separate those out into their components.

----


In [15]:
multi_frames = []
for row in mapping_candidates.itertuples(index=True, name="PandaZ"):
    split_check = row[1]
    if split_check is None:
        split_check = ""
    splitted = pd.Series(split_check.split("|"))
    splitted_count = len(splitted)
    repeated = [split_check] * splitted_count
    repeated = pd.Series(repeated)
    as_frame = pd.DataFrame(dict(repeated=repeated, splitted=splitted)).reset_index()
    seq_list = list(range(1, splitted_count + 1))
    as_frame["part_count"] = splitted_count
    as_frame["seq"] = seq_list
    multi_frames.append(as_frame)
concat_frame = pd.concat(multi_frames)
concat_frame = concat_frame[["repeated", "splitted", "part_count", "seq"]]
concat_frame

,repeated,splitted,part_count,seq
0,mud,mud,1,1
0,marine sediment,marine sediment,1,1
0,sediment,sediment,1,1
0,Sediment,Sediment,1,1
0,deep marine sediment [ENVO:00002113],deep marine sediment [ENVO:00002113],1,1
...,...,...,...,...
0,Sequences,Sequences,1,1
0,Marine sediment,Marine sediment,1,1
0,Lake Sediment,Lake Sediment,1,1
0,Environmental_sample,Environmental_sample,1,1


In [16]:
mapping_candidates = mapping_candidates.merge(
    concat_frame, left_on=biosample_col_to_map, right_on="repeated", how="left"
)

# mapping_candidates

## Normalize a few different ways `OBO` IDs have been entered
In the Biosample metadata

In [17]:
# # this helps by standardizing what could be an ID
# but if the RHS is text, then envo should just be removed
mapping_candidates["term_tidy"] = mapping_candidates.splitted.str.replace(
    primary_onto_prefix.lower() + "[:_ ]",
    primary_onto_prefix + ":",
    regex=True,
    case=False,
)

# mapping_candidates

# Now try to extract ontology terms that are already present

In [18]:
candidate_series_decomposition = scoped_mapping.decompose_series(
    mapping_candidates["term_tidy"], id_patterns[primary_onto_prefix]
)

# candidate_series_decomposition

mapping_candidates = pd.concat(
    [mapping_candidates, candidate_series_decomposition], axis=1
)

mapping_candidates[
    "remaining_tidied"
] = mapping_candidates.remaining_tidied.str.replace(
    primary_onto_prefix.lower() + "[:_ ]", "", regex=True, case=False
)

mapping_candidates

,env_medium,count,repeated,splitted,part_count,seq,term_tidy,string,extract,remaining_string,remaining_tidied
0,mud,3917,mud,mud,1.0,1.0,mud,mud,,mud,mud
1,marine sediment,854,marine sediment,marine sediment,1.0,1.0,marine sediment,marine sediment,,marine sediment,marine sediment
2,sediment,702,sediment,sediment,1.0,1.0,sediment,sediment,,sediment,sediment
3,Sediment,450,Sediment,Sediment,1.0,1.0,Sediment,Sediment,,Sediment,sediment
4,deep marine sediment [ENVO:00002113],185,deep marine sediment [ENVO:00002113],deep marine sediment [ENVO:00002113],1.0,1.0,deep marine sediment [ENVO:00002113],deep marine sediment [ENVO:00002113],ENVO:00002113,deep marine sediment,deep marine sediment
...,...,...,...,...,...,...,...,...,...,...,...
71,Sequences,1,Sequences,Sequences,1.0,1.0,Sequences,Sequences,,Sequences,sequences
72,Marine sediment,1,Marine sediment,Marine sediment,1.0,1.0,Marine sediment,Marine sediment,,Marine sediment,marine sediment
73,Lake Sediment,1,Lake Sediment,Lake Sediment,1.0,1.0,Lake Sediment,Lake Sediment,,Lake Sediment,lake sediment
74,Environmental_sample,1,Environmental_sample,Environmental_sample,1.0,1.0,Environmental_sample,Environmental_sample,,Environmental_sample,environmental sample


## Join the extracted IDs with their labels

_Start by connecting to the rdftab database from which the terms and label-like annotations will be obtained?_

## extracting the labels

In [19]:
q = "select stanza as subject, value from ids_labs_selected_ontolgies"

[onto_labels, query_duration] = scoped_mapping.timed_query(q, biosample_cnx)

onto_labels

,subject,value
0,BFO:0000001,entity
1,BFO:0000002,continuant
2,BFO:0000003,occurrent
3,BFO:0000004,independent continuant
4,BFO:0000006,spatial region
...,...,...
8350,PO:0030132,cincinnus inflorescence
8351,PO:0030133,compound cincinnus inflorescence
8352,PO:0030134,bostryx inflorescence
8353,PO:0030135,drepanium inflorescence


## and merging 

In [20]:
# mapping_candidates.columns

In [21]:
mapping_candidates = mapping_candidates.merge(
    onto_labels, left_on="extract", right_on="subject", how="left"
)
mapping_candidates

,env_medium,count,repeated,splitted,part_count,seq,term_tidy,string,extract,remaining_string,remaining_tidied,subject,value
0,mud,3917,mud,mud,1.0,1.0,mud,mud,,mud,mud,NaN,NaN
1,marine sediment,854,marine sediment,marine sediment,1.0,1.0,marine sediment,marine sediment,,marine sediment,marine sediment,NaN,NaN
2,sediment,702,sediment,sediment,1.0,1.0,sediment,sediment,,sediment,sediment,NaN,NaN
3,Sediment,450,Sediment,Sediment,1.0,1.0,Sediment,Sediment,,Sediment,sediment,NaN,NaN
4,deep marine sediment [ENVO:00002113],185,deep marine sediment [ENVO:00002113],deep marine sediment [ENVO:00002113],1.0,1.0,deep marine sediment [ENVO:00002113],deep marine sediment [ENVO:00002113],ENVO:00002113,deep marine sediment,deep marine sediment,ENVO:00002113,deep marine sediment
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,Sequences,1,Sequences,Sequences,1.0,1.0,Sequences,Sequences,,Sequences,sequences,NaN,NaN
72,Marine sediment,1,Marine sediment,Marine sediment,1.0,1.0,Marine sediment,Marine sediment,,Marine sediment,marine sediment,NaN,NaN
73,Lake Sediment,1,Lake Sediment,Lake Sediment,1.0,1.0,Lake Sediment,Lake Sediment,,Lake Sediment,lake sediment,NaN,NaN
74,Environmental_sample,1,Environmental_sample,Environmental_sample,1.0,1.0,Environmental_sample,Environmental_sample,,Environmental_sample,environmental sample,NaN,NaN


## Use cosine string distance to see if the labels match closely enough

I.e. the labels claimed by the Biosample data set and the labels asserted in the ontology. if they're close enough, consider the assigned ID legit


_How close is close enough?_

In [22]:
my_cosine_obj = Cosine(my_string_dist_arg)
mapping_candidates["value"] = mapping_candidates["value"].fillna("")
mapping_candidates["remaining_tidied"] = mapping_candidates["remaining_tidied"].fillna(
    ""
)
mapping_candidates["cosine"] = mapping_candidates.apply(
    lambda my_row: my_cosine_obj.distance(
        my_row["remaining_tidied"].lower(), my_row["value"].lower()
    ),
    axis=1,
)
mapping_candidates

,env_medium,count,repeated,splitted,part_count,seq,term_tidy,string,extract,remaining_string,remaining_tidied,subject,value,cosine
0,mud,3917,mud,mud,1.0,1.0,mud,mud,,mud,mud,NaN,,1.0
1,marine sediment,854,marine sediment,marine sediment,1.0,1.0,marine sediment,marine sediment,,marine sediment,marine sediment,NaN,,1.0
2,sediment,702,sediment,sediment,1.0,1.0,sediment,sediment,,sediment,sediment,NaN,,1.0
3,Sediment,450,Sediment,Sediment,1.0,1.0,Sediment,Sediment,,Sediment,sediment,NaN,,1.0
4,deep marine sediment [ENVO:00002113],185,deep marine sediment [ENVO:00002113],deep marine sediment [ENVO:00002113],1.0,1.0,deep marine sediment [ENVO:00002113],deep marine sediment [ENVO:00002113],ENVO:00002113,deep marine sediment,deep marine sediment,ENVO:00002113,deep marine sediment,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,Sequences,1,Sequences,Sequences,1.0,1.0,Sequences,Sequences,,Sequences,sequences,NaN,,1.0
72,Marine sediment,1,Marine sediment,Marine sediment,1.0,1.0,Marine sediment,Marine sediment,,Marine sediment,marine sediment,NaN,,1.0
73,Lake Sediment,1,Lake Sediment,Lake Sediment,1.0,1.0,Lake Sediment,Lake Sediment,,Lake Sediment,lake sediment,NaN,,1.0
74,Environmental_sample,1,Environmental_sample,Environmental_sample,1.0,1.0,Environmental_sample,Environmental_sample,,Environmental_sample,environmental sample,NaN,,1.0


**Previously, we did a reality check on the claimed IDs and labels. If a label is claimed without any ID, that could still be a path to an ontology term.**

We'll be doing some merging, so make sure column names are meaningful and unique


In [23]:
mapping_candidates.columns = [
    biosample_col_to_map,
    "count",
    "repeated",
    "splitted",
    "part_count",
    "seq",
    "term_tidy",
    "string",
    "extract",
    "remaining_string",
    "remaining_tidied",
    "term_id",
    "lab_from_id",
    "lfi_cosine",
]
mapping_candidates = mapping_candidates.merge(
    onto_labels, left_on="remaining_tidied", right_on="value", how="left"
)

mapping_candidates.columns = [
    biosample_col_to_map,
    "count",
    "repeated",
    "splitted",
    "part_count",
    "seq",
    "term_tidy",
    "string",
    "extract",
    "remaining_string",
    "remaining_tidied",
    "term_id",
    "lab_from_id",
    "lfi_cosine",
    "term_id_from_lab",
    "value",
]


mapping_candidates

,env_medium,count,repeated,splitted,part_count,seq,term_tidy,string,extract,remaining_string,remaining_tidied,term_id,lab_from_id,lfi_cosine,term_id_from_lab,value
0,mud,3917,mud,mud,1.0,1.0,mud,mud,,mud,mud,NaN,,1.0,ENVO:01000001,mud
1,marine sediment,854,marine sediment,marine sediment,1.0,1.0,marine sediment,marine sediment,,marine sediment,marine sediment,NaN,,1.0,ENVO:03000033,marine sediment
2,sediment,702,sediment,sediment,1.0,1.0,sediment,sediment,,sediment,sediment,NaN,,1.0,ENVO:00002007,sediment
3,Sediment,450,Sediment,Sediment,1.0,1.0,Sediment,Sediment,,Sediment,sediment,NaN,,1.0,ENVO:00002007,sediment
4,deep marine sediment [ENVO:00002113],185,deep marine sediment [ENVO:00002113],deep marine sediment [ENVO:00002113],1.0,1.0,deep marine sediment [ENVO:00002113],deep marine sediment [ENVO:00002113],ENVO:00002113,deep marine sediment,deep marine sediment,ENVO:00002113,deep marine sediment,0.0,ENVO:00002113,deep marine sediment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,Sequences,1,Sequences,Sequences,1.0,1.0,Sequences,Sequences,,Sequences,sequences,NaN,,1.0,NaN,NaN
72,Marine sediment,1,Marine sediment,Marine sediment,1.0,1.0,Marine sediment,Marine sediment,,Marine sediment,marine sediment,NaN,,1.0,ENVO:03000033,marine sediment
73,Lake Sediment,1,Lake Sediment,Lake Sediment,1.0,1.0,Lake Sediment,Lake Sediment,,Lake Sediment,lake sediment,NaN,,1.0,ENVO:00000546,lake sediment
74,Environmental_sample,1,Environmental_sample,Environmental_sample,1.0,1.0,Environmental_sample,Environmental_sample,,Environmental_sample,environmental sample,NaN,,1.0,NaN,NaN


In [24]:
mapping_candidates = mapping_candidates.drop_duplicates()

mc_tifl_split = mapping_candidates["term_id_from_lab"].str.split(":")

import math

prefix_when_possible = []
for one_split in mc_tifl_split:
    if type(one_split) == list:
        os_first = one_split[0]
        prefix_when_possible.append(os_first)
    else:
        prefix_when_possible.append("")

prefix_when_possible = [each_string.lower() for each_string in prefix_when_possible]

# prefix_when_possible

In [25]:
if "onto_prefix" in mapping_candidates.columns:
    mapping_candidates = mapping_candidates.drop("onto_prefix", 1)

mapping_candidates.loc[:, "onto_prefix"] = pd.Series(prefix_when_possible).values

# mapping_candidates

In [26]:
ranks = list(range(len(additional_preferred_ontologies)))

d = {"onto_prefix": additional_preferred_ontologies, "rank": ranks}
onto_rank_frame = pd.DataFrame(d)

onto_rank_frame

,onto_prefix,rank
0,envo,0
1,ncbitaxon,1
2,po,2


In [27]:
mapping_candidates = mapping_candidates.merge(
    onto_rank_frame, on="onto_prefix", how="left"
)

# mapping_candidates

In [28]:
best_maps = mapping_candidates.groupby(biosample_col_to_map)["rank"].min().reset_index()
# best_maps

In [29]:
mapping_candidates = mapping_candidates.merge(
    best_maps, on=[biosample_col_to_map, "rank"]
)
# mapping_candidates

## Find consensus term IDs and labels


In [30]:
mapping_candidates["consensus_id"] = mapping_candidates["term_id_from_lab"]
mapping_candidates["consensus_lab"] = mapping_candidates["value"]

flag = mapping_candidates["consensus_id"].isnull() & (
    (
        ~mapping_candidates["term_id"].isnull()
        & mapping_candidates.lfi_cosine.le(my_max_string_dist)
    )
    | (
        ~mapping_candidates["term_id"].isnull()
        & mapping_candidates["remaining_tidied"].eq("")
    )
)

replacements = mapping_candidates.loc[flag, "term_id"]
mapping_candidates.loc[flag, "consensus_id"] = replacements

replacements = mapping_candidates.loc[flag, "lab_from_id"]
mapping_candidates.loc[flag, "consensus_lab"] = replacements

flag = mapping_candidates.consensus_id.isna()
antiflag = ~flag
mapping_candidates["id_or_lab_ok"] = antiflag

mapping_candidates["assembled_consensus"] = (
    mapping_candidates["consensus_lab"]
    + " ["
    + mapping_candidates["consensus_id"]
    + "]"
)

In [31]:
mapping_candidates

,env_medium,count,repeated,splitted,part_count,seq,term_tidy,string,extract,remaining_string,...,lab_from_id,lfi_cosine,term_id_from_lab,value,onto_prefix,rank,consensus_id,consensus_lab,id_or_lab_ok,assembled_consensus
0,mud,3917,mud,mud,1.0,1.0,mud,mud,,mud,...,,1.0,ENVO:01000001,mud,envo,0.0,ENVO:01000001,mud,True,mud [ENVO:01000001]
1,marine sediment,854,marine sediment,marine sediment,1.0,1.0,marine sediment,marine sediment,,marine sediment,...,,1.0,ENVO:03000033,marine sediment,envo,0.0,ENVO:03000033,marine sediment,True,marine sediment [ENVO:03000033]
2,sediment,702,sediment,sediment,1.0,1.0,sediment,sediment,,sediment,...,,1.0,ENVO:00002007,sediment,envo,0.0,ENVO:00002007,sediment,True,sediment [ENVO:00002007]
3,Sediment,450,Sediment,Sediment,1.0,1.0,Sediment,Sediment,,Sediment,...,,1.0,ENVO:00002007,sediment,envo,0.0,ENVO:00002007,sediment,True,sediment [ENVO:00002007]
4,deep marine sediment [ENVO:00002113],185,deep marine sediment [ENVO:00002113],deep marine sediment [ENVO:00002113],1.0,1.0,deep marine sediment [ENVO:00002113],deep marine sediment [ENVO:00002113],ENVO:00002113,deep marine sediment,...,deep marine sediment,0.0,ENVO:00002113,deep marine sediment,envo,0.0,ENVO:00002113,deep marine sediment,True,deep marine sediment [ENVO:00002113]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,Sequences,1,Sequences,Sequences,1.0,1.0,Sequences,Sequences,,Sequences,...,,1.0,NaN,NaN,,NaN,NaN,NaN,False,NaN
71,Marine sediment,1,Marine sediment,Marine sediment,1.0,1.0,Marine sediment,Marine sediment,,Marine sediment,...,,1.0,ENVO:03000033,marine sediment,envo,0.0,ENVO:03000033,marine sediment,True,marine sediment [ENVO:03000033]
72,Lake Sediment,1,Lake Sediment,Lake Sediment,1.0,1.0,Lake Sediment,Lake Sediment,,Lake Sediment,...,,1.0,ENVO:00000546,lake sediment,envo,0.0,ENVO:00000546,lake sediment,True,lake sediment [ENVO:00000546]
73,Environmental_sample,1,Environmental_sample,Environmental_sample,1.0,1.0,Environmental_sample,Environmental_sample,,Environmental_sample,...,,1.0,NaN,NaN,,NaN,NaN,NaN,False,NaN


## For which Biosample annotations were no easy mappings found?
How many Biosamples uses those annotations?

In [32]:
flag = ~mapping_candidates.id_or_lab_ok
needs_search = mapping_candidates.loc[flag]

needs_search = needs_search[["remaining_tidied", "count"]]

sum_by_needed = needs_search.groupby("remaining_tidied")["count"].sum()

sum_by_needed = sum_by_needed.to_frame()
sum_by_needed["remaining_tidied"] = sum_by_needed.index

sum_by_needed = sum_by_needed.sort_values("count", ascending=False)
sum_by_needed.reset_index(drop=True)

# sum_by_needed.to_sql("sum_by_needed", biosample_cnx, if_exists="replace", index=False)

,count,remaining_tidied
0,106,contaminated sediment
1,72,sediments
2,36,enrichment culture
3,20,cryoconite sediments
4,16,freshwater lake sediment
5,12,genomic dna
6,11,microbial mats
7,10,sediment sample
8,9,wet sediments
9,6,clay layer


In [33]:
# sum_by_needed

## Extract the tidied strings

In [34]:
ebs_raw_list = list(sum_by_needed["remaining_tidied"])
ebs_raw_list.sort()

## Submit those tidied strings to a search engine

Specifically OLS search. This takes roughly one second per unique post-tidied submission

_Turn logging back on to show status?_
_Print the count and pre- and post- datestamps_
`datetime.now().replace(microsecond=0).isoformat()` ?


In [35]:
print(datetime.now().replace(microsecond=0).isoformat())
print(len(sum_by_needed.index))
print((len(sum_by_needed.index)) / 60)

2021-07-04T19:10:17
36
0.6


In [36]:
ebs_search_res = scoped_mapping.search_get_annotations_wrapper(
    ebs_raw_list,
    bad_chars=chars_to_whiteout,
    cat_name=biosample_col_to_map,
    ontoprefix=additional_preferred_ontologies_str,
    query_fields="",
    rr=5,
    string_dist_arg=my_string_dist_arg,
)
# ebs_search_res.to_sql(
#     "ols_search_results", biosample_cnx, if_exists="replace", index=False
# )

## Filter out the best of the acceptable mappings
From a string distance perspective

In [37]:
my_best_acceptable = scoped_mapping.get_best_acceptable(
    ebs_search_res, max_string_dist=my_max_string_dist
)
my_best_acceptable

,category,raw,query,name,string_dist_rank,string_dist,obo_id,label,search_rank,ontology_prefix,scope,type,iri,ontology_name
43,env_medium,crude oil,crude oil,crude oil,1,0.000,ENVO:00002984,petroleum,5,ENVO,has_related_synonym,annotation,http://purl.obolibrary.org/obo/ENVO_00002984,envo
86,env_medium,enrichment culture,enrichment culture,enrichment culture,1,0.000,NCBITaxon:1566338,enrichment culture,1,NCBITAXON,label,label,http://purl.obolibrary.org/obo/NCBITaxon_1566338,ncbitaxon
118,env_medium,microbial mats,microbial mats,microbial mat,1,0.039,ENVO:01000008,microbial mat,1,ENVO,label,label,http://purl.obolibrary.org/obo/ENVO_01000008,envo
257,env_medium,surface sediment,surface sediment,surface sediment,1,0.000,ENVO:01001192,sediment surface,1,ENVO,synonym,synonym,http://purl.obolibrary.org/obo/ENVO_01001192,envo


----

## Filter out the submissions with no acceptable matches

In [38]:
no_acceptable_mappings = scoped_mapping.get_no_acceptable_mappings(
    ebs_search_res, my_best_acceptable
)

no_acceptable_mappings

,category,raw,query,name,string_dist_rank,string_dist,obo_id,label,search_rank,ontology_prefix,scope,type,iri,ontology_name
2,env_medium,anode sediment,anode sediment,anoxic sediment,1,0.259,ENVO:00002045,anaerobic sediment,2,ENVO,synonym,synonym,http://purl.obolibrary.org/obo/ENVO_00002045,envo
0,env_medium,anode sediment,anode sediment,sediment,2,0.266,ENVO:00002007,sediment,1,ENVO,label,label,http://purl.obolibrary.org/obo/ENVO_00002007,envo
3,env_medium,anode sediment,anode sediment,marine sediment,3,0.333,ENVO:03000033,marine sediment,3,ENVO,label,label,http://purl.obolibrary.org/obo/ENVO_03000033,envo
4,env_medium,anode sediment,anode sediment,marine sediments,4,0.355,ENVO:03000033,marine sediment,3,ENVO,synonym,synonym,http://purl.obolibrary.org/obo/ENVO_03000033,envo
6,env_medium,anode sediment,anode sediment,biogenous sediment,5,0.364,ENVO:01000082,biogenous sediment,5,ENVO,label,label,http://purl.obolibrary.org/obo/ENVO_01000082,envo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,env_medium,wet sediments,wet sediments,marine sediment,2,0.383,ENVO:03000033,marine sediment,4,ENVO,label,label,http://purl.obolibrary.org/obo/ENVO_03000033,envo
276,env_medium,wet sediments,wet sediments,wet,3,0.592,PATO:0001823,wet,1,ENVO,label,label,http://purl.obolibrary.org/obo/PATO_0001823,envo
277,env_medium,wet sediments,wet sediments,wet microburst,4,0.760,ENVO:01001756,wet microburst,2,ENVO,label,label,http://purl.obolibrary.org/obo/ENVO_01001756,envo
278,env_medium,wet sediments,wet sediments,wet macroburst,5,0.760,ENVO:01001753,wet macroburst,3,ENVO,label,label,http://purl.obolibrary.org/obo/ENVO_01001753,envo


## Try searching the failures against all ontologies in OLS
what would it be like to use a local OLS instance? (latency-wise)

In [39]:
still_unmapped = list(set(list(no_acceptable_mappings["raw"])))
still_unmapped.sort()

print(len(still_unmapped))

still_unmapped

32


['anode sediment',
 'chimney sample',
 'clay layer',
 'clay loam',
 'contaminated sediment',
 'cryoconite sediments',
 'deep sea surface sediment',
 'dry sediments',
 'dy sediments',
 'environmental sample',
 'freshwater lake sediment',
 'genomic dna',
 'mats',
 'mesophilic anaerobic sludge',
 'salt crust',
 'salt pan crust',
 'sea mud',
 'sediemnt',
 'sediment at spring source',
 'sediment depth 0 1cm',
 'sediment depth 1 2cm',
 'sediment sample',
 'sediment1',
 'sediment2',
 'sediment3',
 'sediments',
 'sequences',
 'solid waste',
 'stainless steel',
 'thermophilic anaerobic sludge',
 'wet sediment',
 'wet sediments']

In [40]:
print(datetime.now().replace(microsecond=0).isoformat())
print(len(sum_by_needed.index))
print((len(sum_by_needed.index)) / 60)

2021-07-04T21:43:47
36
0.6


In [41]:
salvage_search_res = scoped_mapping.search_get_annotations_wrapper(
    still_unmapped,
    bad_chars="._-",
    cat_name="salvage",
    ontoprefix="",
    query_fields="",
    rr=5,
    string_dist_arg=2,
)

# salvage_search_res.to_sql(
#     "salvage_search_res", biosample_cnx, if_exists="replace", index=False
# )

## We appear to be at a point of diminishing returns
At the very least, will require some review

In [42]:
salvage_best_acceptable = scoped_mapping.get_best_acceptable(
    salvage_search_res, max_string_dist=salvage_max_dist
)
salvage_best_acceptable

,category,raw,query,name,string_dist_rank,string_dist,obo_id,label,search_rank,ontology_prefix,scope,type,iri,ontology_name
38,salvage,clay layer,clay layer,clay,1,0.199,ENVO:00002982,clay,1,ENVO,label,label,http://purl.obolibrary.org/obo/ENVO_00002982,envo
88,salvage,contaminated sediment,contaminated sediment,contaminated sediment,1,0.000,ENVO:00002114,contaminated sediment,1,MICRO,label,label,http://purl.obolibrary.org/obo/ENVO_00002114,micro
107,salvage,deep sea surface sediment,deep sea surface sediment,surface sediment,1,0.178,ENVO:01001192,sediment surface,2,ENVO,synonym,synonym,http://purl.obolibrary.org/obo/ENVO_01001192,envo
148,salvage,environmental sample,environmental sample,Cichlidae environmental sample,1,0.177,NCBITaxon:2497813,Cichlidae environmental sample,1,NCBITAXON,label,label,http://purl.obolibrary.org/obo/NCBITaxon_2497813,ncbitaxon
162,salvage,genomic dna,genomic dna,Genomic DNA,1,0.000,NCIT:C95940,Genomic DNA,1,NCIT,label,label,http://purl.obolibrary.org/obo/NCIT_C95940,ncit
175,salvage,mats,mats,mats,1,0.000,NaN,mats,1,CCO,label,label,http://identifiers.org/ncbigene/42634,cco
192,salvage,mesophilic anaerobic sludge,mesophilic anaerobic sludge,anaerobic sludge,1,0.199,ENVO:00002129,anaerobic sludge,1,ENVO,label,label,http://purl.obolibrary.org/obo/ENVO_00002129,envo
418,salvage,sequences,sequences,sequences,1,0.000,data:2044,Sequence,1,EDAM,synonym,synonym,http://edamontology.org/data_2044,edam
431,salvage,solid waste,solid waste,Solid Waste,1,0.000,OMIT:0028125,Solid Waste,1,OMIT,label,label,http://purl.obolibrary.org/obo/OMIT_0028125,omit
439,salvage,stainless steel,stainless steel,Stainless Steel,1,0.000,OMIT:0014071,Stainless Steel,1,OMIT,label,label,http://purl.obolibrary.org/obo/OMIT_0014071,omit


## Save the mappings for review!

_Note that the merge on ID or label mappings are not included here_


In [43]:
my_best_acceptable[include_col] = first_pass_include_val
salvage_best_acceptable[include_col] = salvage_include_val
best_and_salvage = pd.concat([my_best_acceptable, salvage_best_acceptable])

best_and_salvage["ols_curation_notes"] = ""

# best_and_salvage.to_csv(ols_review_file, sep=ols_review_seperator, index=False)

best_and_salvage = best_and_salvage[
    [
        "category",
        "raw",
        "query",
        "name",
        "obo_id",
        "label",
        "ontology_prefix",
        "include",
        "ols_curation_notes",
        "scope",
        "type",
        "iri",
        "ontology_name",
        "string_dist_rank",
        "string_dist",
        "search_rank",
    ]
]

In [44]:
best_and_salvage = best_and_salvage.sort_values(
    ["include", "ontology_prefix"], ascending=(True, True)
)
best_and_salvage

,category,raw,query,name,obo_id,label,ontology_prefix,include,ols_curation_notes,scope,type,iri,ontology_name,string_dist_rank,string_dist,search_rank
175,salvage,mats,mats,mats,NaN,mats,CCO,False,,label,label,http://identifiers.org/ncbigene/42634,cco,1,0.000,1
418,salvage,sequences,sequences,sequences,data:2044,Sequence,EDAM,False,,synonym,synonym,http://edamontology.org/data_2044,edam,1,0.000,1
38,salvage,clay layer,clay layer,clay,ENVO:00002982,clay,ENVO,False,,label,label,http://purl.obolibrary.org/obo/ENVO_00002982,envo,1,0.199,1
107,salvage,deep sea surface sediment,deep sea surface sediment,surface sediment,ENVO:01001192,sediment surface,ENVO,False,,synonym,synonym,http://purl.obolibrary.org/obo/ENVO_01001192,envo,1,0.178,2
192,salvage,mesophilic anaerobic sludge,mesophilic anaerobic sludge,anaerobic sludge,ENVO:00002129,anaerobic sludge,ENVO,False,,label,label,http://purl.obolibrary.org/obo/ENVO_00002129,envo,1,0.199,1
88,salvage,contaminated sediment,contaminated sediment,contaminated sediment,ENVO:00002114,contaminated sediment,MICRO,False,,label,label,http://purl.obolibrary.org/obo/ENVO_00002114,micro,1,0.000,1
148,salvage,environmental sample,environmental sample,Cichlidae environmental sample,NCBITaxon:2497813,Cichlidae environmental sample,NCBITAXON,False,,label,label,http://purl.obolibrary.org/obo/NCBITaxon_2497813,ncbitaxon,1,0.177,1
162,salvage,genomic dna,genomic dna,Genomic DNA,NCIT:C95940,Genomic DNA,NCIT,False,,label,label,http://purl.obolibrary.org/obo/NCIT_C95940,ncit,1,0.000,1
431,salvage,solid waste,solid waste,Solid Waste,OMIT:0028125,Solid Waste,OMIT,False,,label,label,http://purl.obolibrary.org/obo/OMIT_0028125,omit,1,0.000,1
439,salvage,stainless steel,stainless steel,Stainless Steel,OMIT:0014071,Stainless Steel,OMIT,False,,label,label,http://purl.obolibrary.org/obo/OMIT_0014071,omit,1,0.000,1


In [45]:
best_and_salvage.to_clipboard(index=False)

## print mapped column and scoping

In [46]:
print(scoping_col)
print(scoping_value)
print(biosample_col_to_map)

env_package_normalization.EnvPackage
sediment
env_medium


In [47]:
raise SystemExit("Don't skip reviewing and saving!")

SystemExit: Don't skip reviewing and saving!

/Users/MAM/Documents/gitrepos/scoped-mapping/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Do some review in a spreadsheet application!
Specifically, columns `include` and `ols_curation_notes`

Then Save

**Add counts back in to motivate manual lookups if necessary**

----

## Now read the reviewed spreadsheet back in

In [ ]:
# curated = pd.read_csv(ols_review_file, sep=ols_review_seperator)
# curated

## OR copy and paste

In [ ]:
curated = pd.read_clipboard()
curated

## Filter on `include`

In [ ]:
flag = curated["include"]
curated = curated.loc[flag]

curated.columns = "ols_" + curated.columns
# curated.to_sql("curated", biosample_cnx, if_exists="replace", index=False)
# curated

In [ ]:
merge_search_merged = mapping_candidates.merge(
    curated, how="outer", left_on="remaining_tidied", right_on="ols_raw"
)

In [ ]:
flag = (
    ~merge_search_merged["id_or_lab_ok"]
    & ~merge_search_merged["ols_obo_id"].eq("")
    & ~merge_search_merged["ols_obo_id"].isna()
)

replacement = merge_search_merged.loc[flag, "ols_obo_id"]

merge_search_merged.loc[flag, "consensus_id"] = replacement

replacement = merge_search_merged.loc[flag, "ols_label"]

merge_search_merged.loc[flag, "consensus_lab"] = replacement

replacement = (
    merge_search_merged.loc[flag, "consensus_lab"]
    + " ["
    + merge_search_merged.loc[flag, "consensus_id"]
    + "]"
)

merge_search_merged.loc[flag, "assembled_consensus"] = replacement

In [ ]:
# merge_search_merged.to_sql(
#     "merge_search_merged", biosample_cnx, if_exists="replace", index=False
# )

In [ ]:
summary = merge_search_merged[[biosample_col_to_map, "count", "assembled_consensus"]]
summary = summary.sort_values("count", ascending=False)
# summary

In [ ]:
frame_for_sssom = merge_search_merged

na_flag = frame_for_sssom["splitted"].isna()
frame_for_sssom = frame_for_sssom[~na_flag]

na_flag = frame_for_sssom["consensus_id"].isna()
frame_for_sssom = frame_for_sssom[~na_flag]

raw_queries = list(frame_for_sssom["splitted"])
urlencodeds = list(map(urllib.parse.quote, raw_queries))
urlencodeds = [sssom_subject_prefix + ":" + item for item in urlencodeds]

results_rows = len(frame_for_sssom.index)
iso8601_stamp = datetime.now().replace(microsecond=0).isoformat()

sssom_frame = {
    "subject_category": [biosample_col_to_map] * results_rows,
    "subject_label": frame_for_sssom["splitted"],
    "predicate_id": ["skos:relatedMatch"] * results_rows,
    "object_id": frame_for_sssom["consensus_id"],
    "object_label": frame_for_sssom["consensus_lab"],
    "match_type": ["Lexical"] * results_rows,
    "creator_id": ["https://github.com/turbomam/scoped-mapping"] * results_rows,
    #     "mapping_tool": ["https://www.ebi.ac.uk/ols/docs/api"] * results_rows,
    "mapping_date": [iso8601_stamp] * results_rows,
    #     "confidence": 1 - frame_for_sssom["string_dist"],
    "subject_id": urlencodeds,
}

sssom_frame = pd.DataFrame(sssom_frame)

sssom_frame.to_csv(sssom_file, sep=ols_review_seperator, index=False)

# sssom_frame

# From here down requires refactoring and generalization 

That may even be true for some of the blocks above this!

## Join together pipe-delimited env_broad_scale mappings 
_Any other constraints? If part_count is > 1, do we need to check for empty or null env_broad_scale, repeated or splitted?_

In [ ]:
multi_part_flag = merge_search_merged["part_count"] > 1
multi_part_flag.value_counts()

In [ ]:
multi_part_frame = merge_search_merged.loc[multi_part_flag]

multi_part_frame = multi_part_frame[
    [
        biosample_col_to_map,
        "count",
        "repeated",
        "splitted",
        "part_count",
        "seq",
        "consensus_id",
        "consensus_lab",
    ]
]

multi_part_frame = multi_part_frame.sort_values(
    [biosample_col_to_map, "seq"], ascending=(True, True)
)

multi_part_frame

In [ ]:
part_count = multi_part_frame.groupby([biosample_col_to_map], sort=False)[
    "part_count"
].max()
part_count.value_counts()

In [ ]:
seq_max = multi_part_frame.groupby([biosample_col_to_map], sort=False)["seq"].max()
seq_max

In [ ]:
parts_check = pd.DataFrame(dict(part_count=part_count, seq_max=seq_max)).reset_index()

all_parts_flag = parts_check["seq_max"] == parts_check["part_count"]
ebs_with_all_parts = parts_check.loc[all_parts_flag, biosample_col_to_map]
ebs_with_all_parts

all_parts_flag = multi_part_frame[biosample_col_to_map].isin(list(ebs_with_all_parts))
all_parts_frame = multi_part_frame.loc[all_parts_flag]
all_parts_frame

In [ ]:
unique_pipesep_ebs = list(set(list(all_parts_frame[biosample_col_to_map])))
unique_pipesep_ebs.sort()
repipe_dict_list = []
for one_pipesep in unique_pipesep_ebs:
    flag = all_parts_frame[biosample_col_to_map] == one_pipesep
    temp_frame = all_parts_frame.loc[flag]
    ids_list = list(temp_frame["consensus_id"])
    lab_list = list(temp_frame["consensus_lab"])
    s = "|"
    ids_repipe = s.join(ids_list)
    lab_repipe = s.join(lab_list)

    repipe_dict = {
        biosample_col_to_map: one_pipesep,
        "consensus_id": ids_repipe,
        "consensus_lab": lab_repipe,
    }
    repipe_dict_list.append(repipe_dict)
repipe_frame = pd.DataFrame(repipe_dict_list)
repipe_frame

In [ ]:
sssom_min_cols = sssom_frame[["subject_label", "object_id", "object_label"]]

sssom_min_cols.columns = [biosample_col_to_map, "consensus_id", "consensus_lab"]

# sssom_min_cols

In [ ]:
repipe_rows = len(repipe_frame.index)
if repipe_rows > 0:
    direct_and_repipe = pd.concat([sssom_min_cols, repipe_frame])
else:
    direct_and_repipe = sssom_min_cols
# direct_and_repipe

In [ ]:
direct_and_repipe.to_sql(
    "direct_and_repipe", biosample_cnx, if_exists="replace", index=False
)

## refactor this even more?


In [ ]:
q = (
    """
select
	b.id, b."""
    + biosample_col_to_map
    + """,
	dar.consensus_id ,
	dar.consensus_lab
from
	direct_and_repipe dar
left join biosample b on
	b."""
    + biosample_col_to_map
    + """ = dar."""
    + biosample_col_to_map
    + """
inner join biosample_sample_taxon_summary stic on
	b.taxonomy_id = stic.biosample_taxid
join env_package_normalization on
	b.env_package = env_package_normalization.env_package
where """
    + scoping_col
    + " = '"
    + scoping_value
    + "'"
)

[per_biosample_scoped_mapping_results, query_duration] = scoped_mapping.timed_query(
    q, biosample_cnx, print_timing=True
)

In [ ]:
per_biosample_scoped_mapping_results["notes"] = scoping_col + "=" + scoping_value

table_name = "repaired_" + biosample_col_to_map

per_biosample_scoped_mapping_results.to_sql(
    table_name, biosample_cnx, if_exists="append", index=False
)

per_biosample_scoped_mapping_results